In [ ]:
#default_exp geometry

# Geometry

Geometry transforms and projections. Implemented as Tensorflow layers and differentiable if it makes sense.

In [ ]:
#export

import tensorflow as tf
from tensorflow.keras import models, layers
import tensorflow_addons as tfa

import numpy as np

In [ ]:
class PolarToCartesianWarp(layers.Layer):
    """Differentiable Polar Image to Cartersian Mapping"""
    def __init__(self, full=True, scale=1.):
        super(PolarToCartesianWarp, self).__init__()
        self.full = full
        self.scale = scale

    def build(self, input_shape):
        range_bins, angle_bins = input_shape[1:3]
        xx = np.arange(-range_bins, range_bins)/range_bins
        yy = 1 - np.arange(0, range_bins)/range_bins
        
        mg = np.meshgrid(xx, yy)
        
        rr = range_bins - np.sqrt(mg[0]**2 + mg[1]**2) * range_bins
        tt = angle_bins - np.arctan2(mg[1], mg[0])/np.pi*angle_bins

        self.warp = tf.Variable(
            np.stack([tt, rr], axis=-1)[np.newaxis, ...],
            trainable=False,
            dtype=tf.float32
        )

        self.warp = tf.repeat(self.warp, repeats=input_shape[0], axis=0)

    def call(self, inputs):
        return tfa.image.resampler(inputs, self.warp)